# Konfiguration

In [ ]:
QUARTAL = "Abr_YQ-2024-1"

# Pfad zur Abrechnugnsdatei aus EEG Faktura
IMPORT_FILE = "./abrechnung_Abr_YQ-2024-1_export.xlsx"

NAME = "Erneuerbare-Energie-Gemeinschaft XXX"

IBAN = "IBAN der EEG"

BIC = "BIC der EEG"

CREDITOR_ID = "CREDITOR_ID der EEG"

ADDRESS = {
    # The address and all of its fields are optional but in some countries they are required
    "address_type": "ADDR",  # valid: ADDR, PBOX, HOME, BIZZ, MLTO, DLVY
    "department": "Head Office",
    "subdepartment": None,
    "street_name": "Straße X",
    "building_number": "123",
    "postcode": "4234",
    "town": "Bad Ischl",
    "country": "AT",
    "country_subdivision": None,
    "lines": ["Line 1", "Line 2"],
}

## Pakete installieren

In [ ]:
%pip install pandas openpyxl sepaxml

## Excel laden

In [ ]:
import pandas as pd

df = pd.read_excel(IMPORT_FILE, parse_dates=['Datum', 'Empfänger Mandatsausstellung'])

## SEPA Rechnungen Abbucher XML erstellen

In [ ]:
import datetime, uuid
import math


from sepaxml import SepaDD

config = {
    "name": NAME,
    "IBAN": IBAN,
    "BIC": BIC,
    "batch": True,
    "creditor_id": CREDITOR_ID,  # supplied by your bank or financial authority
    "currency": "EUR",  # ISO 4217
    # "instrument": "B2B",  # - default is CORE (B2C)
    "address": ADDRESS
}
sepa_invoices = SepaDD(config, schema="pain.008.001.02", clean=True)

accounting = df[df['Abrechnung'] == QUARTAL]
invoices = accounting[accounting['Dokumenttyp'] == "Rechnung"]
invoices
invoices = invoices[[
    'Nummer', 
    'Empfänger Kontoeigner', 
    'Empfänger Konto IBAN', 
    'Rechnungsbetrag Brutto', 
    'Empfänger Mitgliedsnummer',
    'Empfänger Mandatsausstellung'
]]

invoices['BIC'] = 'NOTPROVIDED'

invoices = invoices.rename(columns={
    'Nummer': 'description',
    'Empfänger Kontoeigner': 'name',
    'Empfänger Konto IBAN': 'IBAN',
    'Rechnungsbetrag Brutto': 'amount',
    'Empfänger Mitgliedsnummer': 'mandate_id',
    'Empfänger Mandatsausstellung': 'mandate_date'
})

# values in CENT!
invoices['amount'] = invoices['amount'] * 100
invoices['amount'] = invoices['amount'].apply(round)
invoices['collection_date'] = datetime.date.today()
invoices['mandate_date'] = invoices['mandate_date'].apply(lambda x: x.to_pydatetime().date())
invoices['mandate_id'] = invoices['mandate_id'].apply(lambda x: str(x).zfill(3))

invoices['type'] = "RCUR"


for index, row in invoices.iterrows():
    payment = row.to_dict()
    sepa_invoices.add_payment(payment)


xml_file = f"{QUARTAL}.xml"

with open(xml_file, 'wb') as file:
    file.write(sepa_invoices.export(validate=True))
    
print(xml_file)


## SEPA Gutschriften XML erstellen

In [ ]:
from sepaxml import SepaTransfer


config = {
    "name": NAME,
    "IBAN": IBAN,
    "BIC": BIC,
    "batch": True,
    "currency": "EUR",  # ISO 4217
    "address": ADDRESS
}

sepa_credits = SepaTransfer(config, clean=True)


accounting = df[df['Abrechnung'] == QUARTAL]
credits = accounting[accounting['Dokumenttyp'] == "Gutschrift"]
credits = credits[[
    'Nummer', 
    'Empfänger Kontoeigner', 
    'Empfänger Konto IBAN', 
    'Rechnungsbetrag Brutto', 
]]
credits['BIC'] = 'NOTPROVIDED'
credits = credits.rename(columns={
    'Nummer': 'description',
    'Empfänger Kontoeigner': 'name',
    'Empfänger Konto IBAN': 'IBAN',
    'Rechnungsbetrag Brutto': 'amount',
})
# values in CENT!
credits['amount'] = credits['amount'] * 100
credits['amount'] = credits['amount'].apply(math.floor)
credits['execution_date'] = datetime.date.today() + datetime.timedelta(days=2)

for index, row in credits.iterrows():
    payment = row.to_dict()
    sepa_credits.add_payment(payment)


xml_file = f"{QUARTAL}.xml"

with open(xml_file, 'wb') as file:
    file.write(sepa_credits.export(validate=True))
    
print(xml_file)
